# Quick Start

In this notebook, we will help you be familiar with Evox through an simple example.

To start with, make sure you have already installed the EvoX.

In [1]:
# install evox, skip it if you have already installed evox
from importlib.util import find_spec

if find_spec("evox") is None:
    %pip install evox

In [2]:
# the dependent packages or functions in this example
import time

import torch

from evox.algorithms import PSO
from evox.problems.numerical import Ackley
from evox.workflows import EvalMonitor, StdWorkflow

## The basic running process in EvoX

As a distributed GPU-accelerated framework for scalable evolutionary computation, EvoX can be uesd to do many kinds computations, so we say "EvoX is all you need". Though many kinds of computaions are different more or less, in EvoX we standard the basic running process:

<div align='center'>
    <b>Initiate an algorithm and a problem -- Set an monitor -- Initiate a workflow -- Run the workflow</b>
</div>


## Initiate an algorithm and a problem

EvoX offers a comprehensive suite of 50+ Evolutionary Algorithms (EAs) and a wide range of 100+ Benchmark Problems/Environments, all benefiting from distributed GPU-acceleration. 

For more details, please refer to our API documentation. [List of Algorithms](https://evox.readthedocs.io/en/latest/api/algorithms/index.html) and [List of Problems](https://evox.readthedocs.io/en/latest/api/problems/index.html).

Here we will create a PSO algorithm and an Ackley function.

In [3]:
# Initiate an algorithm
algorithm = PSO(
    pop_size=100,
    lb=torch.tensor([-10.0]),
    ub=torch.tensor([10.0]),
    w=0.6,
    phi_p=2.5,
    phi_g=0.8,
)

# Initiate a problem
problem = Ackley()

Notice that the algorithm and problem themselves do not contain the monitoring of the steps, so we will not get any feed back only denpending on them. It turns out that we still need a **monitor**.

## Set an monitor

``Monitor`` is a standard class in EvoX to monitor the intermediate values inside a optimization process. Information like fitness or population can be easily obtained by the monitor.

Doing is better than saying, so let us create a “Evalution monitor”:

In [4]:
# Set an monitor
monitor = EvalMonitor()

## Initiate an workflow

A **workflow** outlines the series of steps required to accomplish a task or project. In EvoX, a workflow represents the overall process of evolutionary computation, putting the algorithm, problem, monitor together.

If we want to run the algorithm to solve the problem with a monitor, we need to create a workflow of the `Workflow` class.

In [5]:
# Initiate an workflow
workflow = StdWorkflow()
workflow.setup(
    algorithm=algorithm,
    problem=problem,
    monitor=monitor,
)

## Run the workflow

Now, we can run the workflow:

In [6]:
start = time.time()

# Run the workflow
for i in range(1, 101):
    workflow.step()
    if i % 10 == 0:
        run_time = time.time() - start
        top_fitness = workflow.get_submodule("monitor").topk_fitness
        print(f"The top fitness is {top_fitness} in {run_time:.4f} seconds at the {i}th generation.")

The top fitness is tensor([0.0631]) in 0.1246 seconds at the 10th generation.
The top fitness is tensor([0.0158]) in 0.1291 seconds at the 20th generation.
The top fitness is tensor([0.0026]) in 0.1311 seconds at the 30th generation.
The top fitness is tensor([0.0026]) in 0.1341 seconds at the 40th generation.
The top fitness is tensor([0.0026]) in 0.1361 seconds at the 50th generation.
The top fitness is tensor([0.0026]) in 0.1390 seconds at the 60th generation.
The top fitness is tensor([0.0026]) in 0.1421 seconds at the 70th generation.
The top fitness is tensor([0.0026]) in 0.1441 seconds at the 80th generation.
The top fitness is tensor([0.0026]) in 0.1471 seconds at the 90th generation.
The top fitness is tensor([0.0012]) in 0.1491 seconds at the 100th generation.


Congretulations if you can reproduce the Quick Start demo! Hope you enjoy the EvoX!